<a href="https://colab.research.google.com/github/Antony-gitau/probabilistic_AI_playgraound/blob/main/bayesian_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we are going to tackle an example of bayesian linear regression using pyro, a probabilistic programming tool.

We want to confirm that the terrain ruggedness is related to poorer economic perfomance outside Africa, but reverse effects on income for African countries.  (as that was the conclusion of this [paper](https://diegopuga.org/papers/rugged.pdf))

*We are following an example on pyro webpage.*

In [2]:
%%capture
!pip install -q --upgrade pyro-ppl torch

import pandas as pd
import numpy as np

import pyro
import pyro.distributions as dist



Data:
We will use rugged data in which we only care about:
1. rugged index - quantifies the ruggedness of geographical location in and out of Africa.
2. list of nation in and out of Africa
3. The GDP per capita of the nations for the year 2000 

In [3]:
data_url = "https://d2hg8soec8ck9v.cloudfront.net/datasets/rugged_data.csv"

# We encode the data in the url with the latin-1, character encoding standard
rugged_data = pd.read_csv(data_url, encoding="ISO-8859-1")

In [4]:
rugged_data.head()

,isocode,isonum,country,rugged,rugged_popw,rugged_slope,rugged_lsd,rugged_pc,land_area,lat,...,africa_region_w,africa_region_e,africa_region_c,slave_exports,dist_slavemkt_atlantic,dist_slavemkt_indian,dist_slavemkt_saharan,dist_slavemkt_redsea,pop_1400,european_descent
0,ABW,533,Aruba,0.462,0.380,1.226,0.144,0.000,18.0,12.508,...,0,0,0,0.0,NaN,NaN,NaN,NaN,614.0,NaN
1,AFG,4,Afghanistan,2.518,1.469,7.414,0.720,39.004,65209.0,33.833,...,0,0,0,0.0,NaN,NaN,NaN,NaN,1870829.0,0.0
2,AGO,24,Angola,0.858,0.714,2.274,0.228,4.906,124670.0,-12.299,...,0,0,1,3610000.0,5.669,6.981,4.926,3.872,1223208.0,2.0
3,AIA,660,Anguilla,0.013,0.010,0.026,0.006,0.000,9.0,18.231,...,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,ALB,8,Albania,3.427,1.597,10.451,1.006,62.133,2740.0,41.143,...,0,0,0,0.0,NaN,NaN,NaN,NaN,200000.0,100.0


In [5]:
rugged_data.columns

Index(['isocode', 'isonum', 'country', 'rugged', 'rugged_popw', 'rugged_slope',
       'rugged_lsd', 'rugged_pc', 'land_area', 'lat', 'lon', 'soil', 'desert',
       'tropical', 'dist_coast', 'near_coast', 'gemstones', 'rgdppc_2000',
       'rgdppc_1950_m', 'rgdppc_1975_m', 'rgdppc_2000_m', 'rgdppc_1950_2000_m',
       'q_rule_law', 'cont_africa', 'cont_asia', 'cont_europe', 'cont_oceania',
       'cont_north_america', 'cont_south_america', 'legor_gbr', 'legor_fra',
       'legor_soc', 'legor_deu', 'legor_sca', 'colony_esp', 'colony_gbr',
       'colony_fra', 'colony_prt', 'colony_oeu', 'africa_region_n',
       'africa_region_s', 'africa_region_w', 'africa_region_e',
       'africa_region_c', 'slave_exports', 'dist_slavemkt_atlantic',
       'dist_slavemkt_indian', 'dist_slavemkt_saharan', 'dist_slavemkt_redsea',
       'pop_1400', 'european_descent'],
      dtype='object')

In [6]:
# lets pick three columns we care about (as we stated earlier) and create a dataframe 
ruggedness_dataframe = rugged_data[["cont_africa","rugged","rgdppc_2000"]]
ruggedness_dataframe.head()

,cont_africa,rugged,rgdppc_2000
0,0,0.462,NaN
1,0,2.518,NaN
2,1,0.858,1794.729
3,0,0.013,NaN
4,0,3.427,3703.113


In [7]:
ruggedness_dataframe.isnull().sum()

cont_africa     0
rugged          0
rgdppc_2000    64
dtype: int64

In [8]:
# lets remove the nan values in the rgdppc_2000 column
nan_values = np.isfinite(ruggedness_dataframe.rgdppc_2000)
sum_true = np.sum(nan_values)
sum_false = np.sum(~nan_values)
print("Trues ", sum_true)
print("Falses ", sum_false)

# this is the dataframe before we remove the nan values on the rgdppc_2000 column
print(ruggedness_dataframe.info())

ruggedness_dataframe = ruggedness_dataframe[nan_values]
#this is after we drop the columns
print(ruggedness_dataframe.info())
ruggedness_dataframe.head()

Trues  170
Falses  64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234 entries, 0 to 233
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   cont_africa  234 non-null    int64  
 1   rugged       234 non-null    float64
 2   rgdppc_2000  170 non-null    float64
dtypes: float64(2), int64(1)
memory usage: 5.6 KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 170 entries, 2 to 233
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   cont_africa  170 non-null    int64  
 1   rugged       170 non-null    float64
 2   rgdppc_2000  170 non-null    float64
dtypes: float64(2), int64(1)
memory usage: 5.3 KB
None


,cont_africa,rugged,rgdppc_2000
2,1,0.858,1794.729
4,0,3.427,3703.113
7,0,0.769,20604.460
8,0,0.775,12173.680
9,0,2.688,2421.985


In [9]:
#lets normalize the rgdppc_2000 column 
ruggedness_dataframe['rgdppc_2000'] = np.log(ruggedness_dataframe['rgdppc_2000'])

Lets get started with the bayesian linear regression

In [16]:
import torch
from torch import nn
from pyro.nn import PyroModule


We now train with pytorch optimizers
- adam optimizer
- mean squared error

we start of by capturing the interaction of ruggedness and whether a nation is in africa or not.

Multiplying the values of "count_africa" with "rugged" creates a new feature that represents the combined effect of the two. This is useful in making the model more nuanced as the result of the new feature will show higher values for countries in Africa with rugged terrain which indicates a different effect compared to other combinations.

In [13]:
# we add a feature to capture the interaction btw cont_africa and ruggedness
ruggedness_dataframe['cont_africa_x_ruggedness'] = ruggedness_dataframe['cont_africa'] * ruggedness_dataframe['rugged']


In [17]:
ruggedness_dataframe.columns

Index(['cont_africa', 'rugged', 'rgdppc_2000', 'cont_africa_x_ruggedness'], dtype='object')

In [22]:
print(ruggedness_dataframe.dtypes)

# we convert the ints and float points into tensors
ruggedness_dataframe_in_tensors = torch.tensor(np.array(ruggedness_dataframe[['cont_africa', 'rugged', 'rgdppc_2000', 'cont_africa_x_ruggedness']]))

# we then split the train set from the label set
x_data, y_data = ruggedness_dataframe_in_tensors[:,:-1], ruggedness_dataframe_in_tensors[:,-1]



cont_africa                   int64
rugged                      float64
rgdppc_2000                 float64
cont_africa_x_ruggedness    float64
dtype: object


In [52]:
# the regression model will receive 3 features and output 1 
linear_reg = PyroModule[nn.Linear](3, 1)

# MSE loss and Adam optimizer
loss_fn = torch.nn.MSELoss(reduction='sum')
optimiz = torch.optim.Adam(linear_reg.parameters(), lr=0.005)
num_iter = 1500 

Notes on the train function:
- squeezing the tensor along the dimension -1 removes any singleton dimension present in the output tensor. That is what the .squeeze(-1) is doing.

- forward pass on the model involves multiplying the input features by the model's weights and applying any defined activation functions or biases.

- Zero grad ensures that the gradients from the previous iterations of training do not accumulate and affect the current iteration's computations.

In [53]:
def train():
  y_pred = linear_reg(x_data).squeeze(-1)
  #print(y_pred.dtype)
  #y_pred = linear_reg(x_data.to(linear_reg.weight.dtype)).squeeze(-1)
  loss_calc = loss_fn(y_pred, y_data)
  # set the gradients to zero for the upcoming iteration
  optimiz.zero_grad
  loss_calc.backward()
  optimiz.step()
  return loss_calc



In [60]:
for iter in range(num_iter):
  loss_calc = train()

  # we print in the iterations of 50
  if (iter + 1) % 50 == 0:
    print("Iteration %04d, loss %.4f" %(iter + 1, loss_calc.item()))

RuntimeError: ignored